In [1]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification, AutoModelForSequenceClassification
from transformers import pipeline
import evaluate
from evaluate import evaluator
import json
import pandas as pd
from datasets import Dataset


/u/athbagde/anaconda3/envs/CL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-17 12:25:31.956456: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
f = open ('/u/athbagde/ML_CL/EXIST 2023 Dataset 2/dev/EXIST2023_dev.json', "r")
  
# Reading from file
data = json.loads(f.read())

In [3]:
val_training_set  = pd.DataFrame(columns=['id','text','lang','sex','age','s_label','t2_lb','t3_lb'])

In [4]:
for id in data:
    sample = data[id]
    val_training_set.loc[len(val_training_set.index)] = [sample['id_EXIST'],sample['tweet'],
                                                sample['lang'],sample['gender_annotators'],
                                                sample['age_annotators'],sample['labels_task1'],
                                                sample['labels_task2'],sample['labels_task3']
                                                ]

In [5]:
val_data =  val_training_set.set_index(['id','text','lang']).apply(pd.Series.explode).reset_index()

In [6]:
val_set = val_data[['text','s_label']]
map_dict = {"YES":1,"NO":0}
val_set['label'] = val_set['s_label'].map(map_dict)
val_set = val_set[['text','label']]
val_ds = Dataset.from_pandas(val_set)

/tmp/ipykernel_118175/3758193207.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set['label'] = val_set['s_label'].map(map_dict)


In [8]:
val_set

,text,label
0,@Fichinescu La comunidad gamer es un antro de ...,0
1,@Fichinescu La comunidad gamer es un antro de ...,1
2,@Fichinescu La comunidad gamer es un antro de ...,1
3,@Fichinescu La comunidad gamer es un antro de ...,0
4,@Fichinescu La comunidad gamer es un antro de ...,1
...,...,...
6223,@ElDukemane You look like a whore in ur new pi...,1
6224,@ElDukemane You look like a whore in ur new pi...,1
6225,@ElDukemane You look like a whore in ur new pi...,1
6226,@ElDukemane You look like a whore in ur new pi...,1


In [9]:
tokenizer = AutoTokenizer.from_pretrained('/u/athbagde/ML_CL/model1')

In [10]:
model =  AutoModelForSequenceClassification.from_pretrained('/u/athbagde/ML_CL/model1')
metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [11]:
task_evaluator = evaluator("text-classification")
eval_results = task_evaluator.compute(
    model_or_pipeline="/u/athbagde/ML_CL/model1",
    data=val_ds,
    label_mapping={"LABEL_0": 0, "LABEL_1": 1},
    tokenizer=tokenizer,
    metric= metrics
)

In [12]:
print(eval_results)

{'accuracy': 0.7459858702633269, 'f1': 0.7333108563722184, 'precision': 0.7413087934560327, 'recall': 0.7254836557705137, 'total_time_in_seconds': 70.26742511498742, 'samples_per_second': 88.63281940114273, 'latency_in_seconds': 0.011282502426940818}


## Evaluating using different val_sets

### ES only model

In [13]:
val_set = val_data.loc[val_data['lang']=='es']
val_set = val_set[['text','s_label']]
map_dict = {"YES":1,"NO":0}
val_set['label'] = val_set['s_label'].map(map_dict)
val_set = val_set[['text','label']]
val_ds = Dataset.from_pandas(val_set)

In [14]:
model =  AutoModelForSequenceClassification.from_pretrained('/u/athbagde/ML_CL/es_only_model1')
tokenizer = AutoTokenizer.from_pretrained('/u/athbagde/ML_CL/es_only_model1')

In [15]:
task_evaluator = evaluator("text-classification")
eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    data=val_ds,
    label_mapping={"LABEL_0": 0, "LABEL_1": 1},
    tokenizer=tokenizer,
    metric= metrics
)

In [16]:
print(eval_results)

{'accuracy': 0.7240437158469946, 'f1': 0.7297056199821589, 'precision': 0.7329749103942652, 'recall': 0.7264653641207816, 'total_time_in_seconds': 36.6687402180396, 'samples_per_second': 89.83128355141798, 'latency_in_seconds': 0.011131979422598543}


## EN only model

In [17]:
val_set = val_data.loc[val_data['lang']=='en']
val_set = val_set[['text','s_label']]
map_dict = {"YES":1,"NO":0}
val_set['label'] = val_set['s_label'].map(map_dict)
val_set = val_set[['text','label']]
val_ds = Dataset.from_pandas(val_set)

In [18]:
val_set

,text,label
3294,"@Mike_Fabricant “You should smile more, love. ...",0
3295,"@Mike_Fabricant “You should smile more, love. ...",0
3296,"@Mike_Fabricant “You should smile more, love. ...",0
3297,"@Mike_Fabricant “You should smile more, love. ...",0
3298,"@Mike_Fabricant “You should smile more, love. ...",1
...,...,...
6223,@ElDukemane You look like a whore in ur new pi...,1
6224,@ElDukemane You look like a whore in ur new pi...,1
6225,@ElDukemane You look like a whore in ur new pi...,1
6226,@ElDukemane You look like a whore in ur new pi...,1


In [19]:
model =  AutoModelForSequenceClassification.from_pretrained('/u/athbagde/ML_CL/en_only_model1')
tokenizer = AutoTokenizer.from_pretrained('/u/athbagde/ML_CL/en_only_model1')

In [20]:
task_evaluator = evaluator("text-classification")
eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    data=val_ds,
    label_mapping={"LABEL_0": 0, "LABEL_1": 1},
    tokenizer=tokenizer,
    metric= metrics
)

In [21]:
print(eval_results)

{'accuracy': 0.7706203135650989, 'f1': 0.7416506717850287, 'precision': 0.7453703703703703, 'recall': 0.7379679144385026, 'total_time_in_seconds': 32.353457474848256, 'samples_per_second': 90.68582553443962, 'latency_in_seconds': 0.011027081620602676}


### M only model

In [22]:
val_set = val_data.loc[val_data['sex']=='M']
val_set = val_set[['text','s_label']]
map_dict = {"YES":1,"NO":0}
val_set['label'] = val_set['s_label'].map(map_dict)
val_set = val_set[['text','label']]
val_ds = Dataset.from_pandas(val_set)

In [23]:
val_set

,text,label
3,@Fichinescu La comunidad gamer es un antro de ...,0
4,@Fichinescu La comunidad gamer es un antro de ...,1
5,@Fichinescu La comunidad gamer es un antro de ...,0
9,@anacaotica88 @MordorLivin No me acuerdo de lo...,1
10,@anacaotica88 @MordorLivin No me acuerdo de lo...,1
...,...,...
6220,@DawnAnd91320913 I haven't seen anything that ...,1
6221,@DawnAnd91320913 I haven't seen anything that ...,1
6225,@ElDukemane You look like a whore in ur new pi...,1
6226,@ElDukemane You look like a whore in ur new pi...,1


In [24]:
model =  AutoModelForSequenceClassification.from_pretrained('/u/athbagde/ML_CL/M_only_model1')
tokenizer = AutoTokenizer.from_pretrained('/u/athbagde/ML_CL/M_only_model1')

In [25]:
task_evaluator = evaluator("text-classification")
eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    data=val_ds,
    label_mapping={"LABEL_0": 0, "LABEL_1": 1},
    tokenizer=tokenizer,
    metric= metrics
)

In [27]:
print(eval_results)

{'accuracy': 0.7392421323057161, 'f1': 0.7271505376344086, 'precision': 0.7213333333333334, 'recall': 0.7330623306233063, 'total_time_in_seconds': 35.0978658630047, 'samples_per_second': 88.72334324128656, 'latency_in_seconds': 0.011270990964356036}


### F only model

In [28]:
val_set = val_data.loc[val_data['sex']=='F']
val_set = val_set[['text','s_label']]
map_dict = {"YES":1,"NO":0}
val_set['label'] = val_set['s_label'].map(map_dict)
val_set = val_set[['text','label']]
val_ds = Dataset.from_pandas(val_set)

In [29]:
val_set

,text,label
0,@Fichinescu La comunidad gamer es un antro de ...,0
1,@Fichinescu La comunidad gamer es un antro de ...,1
2,@Fichinescu La comunidad gamer es un antro de ...,1
6,@anacaotica88 @MordorLivin No me acuerdo de lo...,1
7,@anacaotica88 @MordorLivin No me acuerdo de lo...,1
...,...,...
6217,@DawnAnd91320913 I haven't seen anything that ...,1
6218,@DawnAnd91320913 I haven't seen anything that ...,1
6222,@ElDukemane You look like a whore in ur new pi...,1
6223,@ElDukemane You look like a whore in ur new pi...,1


In [30]:
model =  AutoModelForSequenceClassification.from_pretrained('/u/athbagde/ML_CL/F_only_model1')
tokenizer = AutoTokenizer.from_pretrained('/u/athbagde/ML_CL/F_only_model1')

In [31]:
task_evaluator = evaluator("text-classification")
eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    data=val_ds,
    label_mapping={"LABEL_0": 0, "LABEL_1": 1},
    tokenizer=tokenizer,
    metric= metrics
)

In [32]:
print(eval_results)

{'accuracy': 0.75208734746307, 'f1': 0.7504848093083387, 'precision': 0.7385496183206107, 'recall': 0.7628120893561103, 'total_time_in_seconds': 35.31043099309318, 'samples_per_second': 88.18923792261577, 'latency_in_seconds': 0.011339252085129472}
